In [179]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive, fixed, interact_manual, Layout, HBox, VBox,IntSlider

In [180]:
def BWOrderBatching(mu,std,R,N):
    r = np.ceil(stats.norm(mu,std).rvs(size=N))
    df = pd.DataFrame({'Store Demand':r})
    df.index +=1
    df['Order Daily'] = df['Store Demand']
    df['Order EOD'] = df['Order Daily'].rolling(2).sum()
    df.iloc[::2,2] = 0
    df['Order Every '+str(R)] = df['Order Daily'].rolling(R).sum()
    df.loc[~((df.index)%R==0),'Order Every '+str(R)] = 0
    fig,ax = plt.subplots(2,1,figsize=(20,10));
    df.replace(np.nan,0).plot(ax=ax[0]);
    ax[0].legend(loc='upper left');
    df.boxplot(ax=ax[1]);
    plt.show()
    display(np.round((df.var(axis=0)).to_frame('Variance').join(df.describe().transpose()),2));
    qgrid_widget = qgrid.show_grid(np.round(df, 2), show_toolbar=True, grid_options={
    'forceFitColumns': True, 'defaultColumnWidth': 200})
    display(qgrid_widget)

# Remove semicolon if you want to inspect the dataframe
qgrid_widget

bwhip = interactive(BWOrderBatching, {'manual': True, 'manual_name': 'Create Figure'}, 
    mu=IntSlider(min=10,description='mu',max=1000,step=10,value=100), 
    std=IntSlider(min=0,description='sigma',max=200,step=1,value=10),
    R=IntSlider(min=1,description='R',max=10,step=1,value=7),
    N=IntSlider(min=1,description='N',max=500,step=1,value=14))

controls = HBox(bwhip.children[:-1], layout=Layout(flex_flow='row wrap'))
output = bwhip.children[-1]
display(VBox([controls, output]))